In [ ]:
%load_ext autoreload
%autoreload 2


import sys, os
sys.path.insert(0, './python_src/')
from IPython.display import Image, display

import numpy as np
import pandas as pd
import pickle

import load_proteins as load
import protein_algs as palgs


import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from pymol import cmd


Tasks
- Try running full analysis for GCK and see if result spreadsheat is the same
- pfkA resturns error in topo strain pathway analysis
- put more code into python library
    - maybe one centralized python source file that then loads multiple other files
    - load protein dictionary object
    - analyze topology - maybe return full skeletons
    - analyze skeletons - return pathways and hinges
    - display routines
- return all proteins and check results
- check proteins in check special function
- create environment configuration file
- check citations for list of proteins
- do not use pure proximal contacts


In [ ]:
df_db = pd.read_excel('data/proteins.xlsx', sheet_name='allosteric')

df_db.fillna("", inplace=True)

df_db

In [ ]:
exclude_bond_types=[('proximal', '')]


selection = np.arange(16, 30)

for index, row in df_db.iterrows():
    
    if row['skip'] == 'yes':
        continue

    if index not in selection:
        continue

    
    prot_id = row['protein_id']
    
    print(index, prot_id)
    
    display(row)

    
    iPDB_id = row['inactive_PDB']
    ichain_list = row['inactive_chains']
    load.preprocess(prot_id, iPDB_id, check=True)
    
    aPDB_id = row['active_PDB']
    achain_list = row['active_chains']
    load.preprocess(prot_id, aPDB_id, check=True)

    reg_list = row['regulatory_ids']
    sub_list = row['substrate_ids']
    
    print(reg_list, sub_list)
    
    idf_prot, idf_bonds = load.load_protein(prot_id, iPDB_id, ichain_list, reg_list, sub_list, exclude_bond_types=exclude_bond_types)
    adf_prot, adf_bonds = load.load_protein(prot_id, aPDB_id, achain_list, reg_list, sub_list, exclude_bond_types=exclude_bond_types)

    display(idf_prot)
    display(adf_prot)
    

    # if activated then choose active configuration as reference
    # this makes the topological features prettier
    if row['mechanism'] == 'activated':
        df_prot_ref = adf_prot
        df_bonds_ref = adf_bonds
        df_prot_def = idf_prot
        df_bonds_def = idf_bonds
    else:
        df_prot_ref = idf_prot
        df_bonds_ref = idf_bonds
        df_prot_def = adf_prot
        df_bonds_def = adf_bonds
        
    
    df_prot_merged = palgs.merge_structures(df_prot_ref, df_prot_def)
    df_bonds_ref_merged = palgs.merge_bonds(df_prot_ref, df_bonds_ref, df_prot_merged)
    
    
    print("Merged Protein Structure:")
    display(df_prot_merged)
    
    print("Merged Reference Bonds:")
    display(df_bonds_ref_merged)
    
    palgs.df_to_pdb(prot_id, df_prot_ref, label='full_reference')
    palgs.df_to_pdb(prot_id, df_prot_def, label='full_deformed')
    
    palgs.df_to_pdb(prot_id, df_prot_merged, suffix='_ref', label='merged_reference')
    palgs.df_to_pdb(prot_id, df_prot_merged, suffix='_def', label='merged_deformed')
    
     
    with open("data/" + prot_id + "/structure.pkl", 'wb') as pkl_file:
        data = {'reference structure': df_prot_ref, 'reference bonds': df_bonds_ref,
               'deformed structure': df_prot_def, 'deformed bonds': df_bonds_def,
               'merged structure': df_prot_merged, 'merged reference bonds': df_bonds_ref_merged}
        pickle.dump(data, pkl_file)
        
    
    

In [ ]:
# selection = np.arange(51)

for index, row in df_db.iterrows():
        
    if row['skip'] == 'yes':
        continue

    if index not in selection:
        continue
        
    print(row['protein_id'])

    cmd.reinitialize()
    cmd.run("python_src/pymol_cmds.py")
    cmd.do("show_disp {}".format(index))
    cmd.ray(800, 800)
    cmd.png("data/figs/test_disp.png")

    
    fig, ax1 = plt.subplots(1, 1, constrained_layout=True, figsize=(6, 6))
    
    ax1.imshow(mpimg.imread('data/figs/test_disp.png'))
    
    ax1.get_xaxis().set_ticks([])
    ax1.get_yaxis().set_ticks([])
    
    plt.show()
    

In [ ]:
# selection = [53]

if os.path.exists("data/results.xlsx"):
    df_results =  pd.read_excel('data/results.xlsx')

else:
    df_results = pd.DataFrame(columns=["protein_id", "subunit_structure", "lrmsd_norm", 
                                       "hinge_scale", "overlap", "sector_sizes", 
                                       "multi_hinge_scale", "multi_overlap", "multi_sector_sizes",
                                      "allo_path_scale", "allo_path_norm", 
                                       "coop_path_scale", "coop_path_norm"])

df_results.set_index("protein_id", inplace=True)

df_results['sector_sizes'] = df_results['sector_sizes'].astype('object')
df_results['multi_sector_sizes'] = df_results['multi_sector_sizes'].astype('object')

display(df_results)
    
l0 = 15

for index, row in df_db.iterrows():
    
    if row['skip'] == 'yes':
        continue

    if index not in selection:
        continue
    
    
    N_sectors = df_db.loc[index, 'n_sectors']
    
    
#     if N_sectors == 2:
#         continue
    
    prot_id = row['protein_id']
    
    print("######################################################")
        
    print(index, prot_id)
    
#     df_results.drop(index=prot_id, inplace=True, errors='ignore')
    
    df_results.loc[prot_id, 'subunit_structure'] = row['subunit_structure']
    
    with open("data/" + prot_id + "/structure.pkl", 'rb') as pkl_file:
        data = pickle.load(pkl_file)
        df_prot_merged = data['merged structure']
        df_bonds_ref_merged = data['merged reference bonds']
        
        df_prot_ref = data['reference structure']
        df_bonds_ref = data['reference bonds']
        
    print("Protein Size:", len(df_prot_merged.index), flush=True)
    
    # calculate local deformation
    palgs.calc_local_rmsd(df_prot_merged, l0=l0)
      
    
#     edgei, edgej = palgs.find_edges(df_prot_ref, df_bonds_ref)
        
        
    has_allo_path = False
    has_coop_path = False
    # find allosteric pathways
    if len(df_prot_merged.query("allo_site!=-1 and active_site==-1")) > 0 and len(df_prot_merged.query("allo_site==-1 and active_site!=-1")) > 0:
        has_allo_path = True
    
        allo_strain_path_scales, allo_strain_path_lengths, allo_strain_paths, max_edge_scale = palgs.find_allo_strain_path(df_prot_merged, df_bonds_ref_merged)
        allo_strain_path_scale = allo_strain_path_scales[0]
        allo_strain_path = allo_strain_paths[0]
                
        print("Allo Path Scale:", allo_strain_path_scale)
        
        df_results.loc[prot_id, 'allo_path_scale'] = allo_strain_path_scale
           
        allo_path_norm = df_prot_merged.iloc[allo_strain_path[0]]['lrmsd']
        
        df_results.loc[prot_id, 'allo_path_norm'] = allo_path_norm
        
        print("Allo Path Norm:", allo_path_norm)
            
        df_prot_merged['allo_path'] = -1
        for i, vi in enumerate(allo_strain_path):
            df_prot_merged.iloc[vi, df_prot_merged.columns.get_loc("allo_path")] = i
            
        df_bonds_ref_merged['allo_path_max_scale'] = max_edge_scale
            

    # find cooperative pathways
    if len(df_prot_merged.query("allo_site==-1 and active_site!=-1")['active_site'].unique()) > 1:
        has_coop_path = True
        
        coop_strain_path_scales, coop_strain_path_lengths, coop_strain_paths, max_edge_scale = palgs.find_coop_strain_path(df_prot_merged, df_bonds_ref_merged)
        coop_strain_path_scale = coop_strain_path_scales[0]
        coop_strain_path = coop_strain_paths[0]
        
        print("Coop Path Scale:", coop_strain_path_scale)
    
        df_results.loc[prot_id, 'coop_path_scale'] = coop_strain_path_scale
        
        coop_path_norm = np.max([df_prot_merged.iloc[coop_strain_path[0]]['lrmsd'], df_prot_merged.iloc[coop_strain_path[-1]]['lrmsd']])
        
        df_results.loc[prot_id, 'coop_path_norm'] = coop_path_norm
        
        print("Coop Path Norm:", coop_path_norm)
            
        
        df_prot_merged['coop_path'] = -1
        for i, vi in enumerate(coop_strain_path):
            df_prot_merged.iloc[vi, df_prot_merged.columns.get_loc("coop_path")] = i
            
        df_bonds_ref_merged['coop_path_max_scale'] = max_edge_scale
    
    if len(df_prot_merged.query("allo_site!=-1")) > 0:
        S_max = df_prot_merged.query("allo_site!=-1")['lrmsd'].max()
        print("S_max:", S_max)
        df_results.loc[prot_id, 'S_max'] = S_max

        S_avg = df_prot_merged.query("allo_site!=-1")['lrmsd'].mean()
        print("S_avg:", S_avg)
        df_results.loc[prot_id, 'S_avg'] = S_avg
        
    if len(df_prot_merged.query("active_site!=-1")) > 0:
        T_max = df_prot_merged.query("active_site!=-1")['lrmsd'].max()
        print("T_max:", T_max)
        df_results.loc[prot_id, 'T_max'] = T_max
        
        T_avg = df_prot_merged.query("active_site!=-1")['lrmsd'].mean()
        print("T_avg:", T_avg)
        df_results.loc[prot_id, 'T_avg'] = T_avg
        

    if has_allo_path:
        print("Using allosteric pathway normalization.")
        df_results.loc[prot_id, 'lrmsd_norm'] = df_results.loc[prot_id, 'allo_path_norm']
    elif has_coop_path:
        print("Using cooperative pathway normalization.")
        df_results.loc[prot_id, 'lrmsd_norm'] = df_results.loc[prot_id, 'coop_path_norm'] 
    # only has allosteric sites labeled
    elif len(df_prot_merged.query("allo_site!=-1")) > 0:
        print("Using allosteric site normalization.")
        df_results.loc[prot_id, 'lrmsd_norm'] = df_results.loc[prot_id, 'S_avg']
    # only has active sites labeled
    elif len(df_prot_merged.query("active_site!=-1")) > 0:
        print("Using active site normalization.")
        df_results.loc[prot_id, 'lrmsd_norm'] = df_results.loc[prot_id, 'T_avg']
    else:
        print("Error finding normalization...")
        
    print("Normalization:", df_results.loc[prot_id, 'lrmsd_norm'])
    
    min_size = 200
    
    print("Min Sector Size:", min_size)
        
    hinge_scale, hinge_overlap, sectors_to_verts = palgs.find_hinge(df_prot_merged, df_bonds_ref_merged, N_sectors=2, min_size=min_size)
    
    print("Hinge Scale:", hinge_scale)
    print("Overlap:", hinge_overlap)
    
    
    sector_sizes = [len(sectors_to_verts[si]) for si in range(len(sectors_to_verts))]
    
        
    df_results.loc[prot_id, 'hinge_scale'] = hinge_scale
    df_results.loc[prot_id, 'overlap'] = hinge_overlap
    df_results.at[prot_id, 'sector_sizes'] = sector_sizes
    
    df_prot_merged['sector'] = -1
    for si in range(len(sectors_to_verts)):
        df_prot_merged.iloc[list(sectors_to_verts[si]), df_prot_merged.columns.get_loc("sector")] = si
    
        
    if N_sectors > 2:
        
        multi_hinge_scale, multi_hinge_overlap, sectors_to_verts = palgs.find_hinge(df_prot_merged, df_bonds_ref_merged, N_sectors=N_sectors, min_size=min_size)

        sector_sizes = [len(sectors_to_verts[si]) for si in range(len(sectors_to_verts))]
        
        print("Multi Hinge Scale:", multi_hinge_scale)
        print("Multi Overlap:", multi_hinge_overlap)

        df_results.loc[prot_id, 'multi_hinge_scale'] = multi_hinge_scale
        df_results.loc[prot_id, 'multi_overlap'] = multi_hinge_overlap
        df_results.at[prot_id, 'multi_sector_sizes'] = sector_sizes
        
        df_prot_merged['multi_sector'] = -1
        for si in range(len(sectors_to_verts)):
            df_prot_merged.iloc[list(sectors_to_verts[si]), df_prot_merged.columns.get_loc("multi_sector")] = si

    
   
    
    with open("data/" + prot_id + "/structure.pkl", 'wb') as pkl_file:
        data['merged structure'] = df_prot_merged
        data['merged reference bonds'] = df_bonds_ref_merged
        pickle.dump(data, pkl_file)
        
    print("######################################################")
        
    print("Normalized Hinge Scale:", hinge_scale/df_results.loc[prot_id, 'lrmsd_norm'])
    print("Hinge Overlap:", hinge_overlap)
    
    if N_sectors > 2:
        print("Normalized Multidomain Hinge Scale:", multi_hinge_scale/df_results.loc[prot_id, 'lrmsd_norm'])
        print("Multidomain Hinge Overlap:", multi_hinge_overlap)
    
    if has_allo_path:
        print("Normalized Allo Pathway Scale:", allo_strain_path_scale/df_results.loc[prot_id, 'lrmsd_norm'])
    if has_coop_path:
        print("Normalized Coop Pathway Scale:", coop_strain_path_scale/df_results.loc[prot_id, 'lrmsd_norm'])
        
    print("######################################################")
        
    display(df_results.loc[prot_id])
        
    df_results.to_excel("data/results.xlsx") 
        
display(df_results)
    

In [ ]:
# selection = [53]

for index, row in df_db.iterrows():
        
    if row['skip'] == 'yes':
        continue

    if index not in selection:
        continue
        
    print(index, row['protein_id'])
    
#     print(df_results.loc[row['protein_id']])

    cmd.reinitialize()
    cmd.run("python_src/pymol_cmds.py")
    cmd.do("show_disp {}".format(index))
    cmd.ray(800, 800)
    cmd.png('data/figs/{}_disp.png'.format(row['protein_id']))

    cmd.reinitialize()
    cmd.run("python_src/pymol_cmds.py")
    cmd.do("show_topo {}".format(index))
    cmd.ray(800, 800)
    cmd.png('data/figs/{}_topo.png'.format(row['protein_id']))
    
    # cmd.reinitialize()
    # cmd.run("python_src/pymol_cmds.py")
    # cmd.do("show_paths {}".format(index))
    # cmd.ray(800, 800)
    # cmd.png('data/figs/{}_topo.png'.format(row['protein_id']))
    
    
    fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, figsize=(12, 6))
    
    ax1.imshow(mpimg.imread('data/figs/{}_disp.png'.format(row['protein_id'])))
    
    ax1.get_xaxis().set_ticks([])
    ax1.get_yaxis().set_ticks([])
    
    ax2.imshow(mpimg.imread('data/figs/{}_topo.png'.format(row['protein_id'])))
    
    ax2.get_xaxis().set_ticks([])
    ax2.get_yaxis().set_ticks([])
    
    plt.show()
    
    

